In [65]:
import pandas as pd


mcca2 = pd.read_csv("multicca2.csv", sep=",")
mcca1 = pd.read_csv("multicca1.csv", sep=",")


In [3]:
import numpy as np

In [5]:
import pyomo.environ as pyo

In [82]:
datasets = [mcca1.iloc[:,1:7].values.tolist(), mcca2.iloc[:,1:6].values.tolist()]

In [67]:
standardize = True
mimic_R = True
# penalties usually same length as datasets
penalties = 1
featurs = len(datasets[0][0])

In [93]:
def ObjRule(model):
    """Objective Function (4.3 in witten 2009)"""
    # tuple(tuples)
    
    return sum((model.w[xi].T @ model.X[idx].T  @ model.X[jdx] @ model.w[xj]) for idx, xi in enumerate(model.X) for jdx, xj in enumerate(model.X) if idx<jdx )

In [9]:
def lasso(model, xi):
    # sum over all entries of vector wi (i in [1:K])
    return sum(abs(model.w[xi][f]) for f in range(len(model.w[xi])) ) <= model.c[xi]

In [10]:
def norm2(model, xi):
    # sum over all entries of vector wi (i in [1:K])
    return sum(model.w[xi][f] * model.w[xi][f] for f in range(len(model.w[xi])) <= 1)

In [62]:
def init_w(xi):
    # TODO use svd vh from scipy 
    return np.zeros(featurs)
    #return [0] * featurs

In [12]:
def scale(mtx, center=True, scale=True):
    """
    Reimplement scale function from R
    """
    if not center:
        raise NotImplementedError('Scaling without centering not implemented')

    centered = mtx - np.mean(mtx, axis=0)
    if not scale:
        return centered

    # to replicate the R implementation of scale, we apply Bessel's
    # correction when calculating the standard deviation in numpy
    scaled = centered / centered.std(axis=0, ddof=1)
    return scaled


In [368]:
'''
# TODO: changes list to numpy and then set doesn't work

# preprocess data
datasets = datasets.copy()
# 2 features needed
for data in datasets:
    if len(data[0]) < 2:
        raise Exception('Need at least 2 features in each dataset')

    # standardize if set TRUE
if standardize:
    for idx in range(len(datasets)):
        if mimic_R:
            datasets[idx] = scale(datasets[idx], center=True, scale=True)
        else:
            datasets[idx] = scale(datasets[idx], center=True, scale=False)

'''

In [84]:
datasets_tuples = [tuple(map(tuple,data)) for data in datasets]

In [78]:
datasets_tuples

[((-0.334610924204471,
   -0.97306366867377,
   -0.075583651455657,
   -0.153609815415446,
   -0.851993676462297),
  (5.55820588710131,
   -3.93991158036192,
   -6.71995778712731,
   5.28723385899056,
   -0.592665154219762),
  (-1.41301591985501,
   1.22647353788828,
   2.84067502332716,
   0.558083501810207,
   4.77974893277835),
  (-6.21790636329919,
   -0.855823906305692,
   4.28703572371224,
   -3.52227004424225,
   2.55882984237342),
  (5.13895331423074,
   1.21981618321478,
   -0.190048289107061,
   -0.195436562711887,
   1.36919618440577),
  (-8.64338469215024,
   -1.84681258887914,
   -0.738421117287049,
   5.89215357987772,
   -7.57379581603491),
  (-9.68340523351363,
   -6.97307664418281,
   -6.82514186354428,
   5.18311020964875,
   0.245243568811294),
  (1.91891828264879,
   1.18361082442049,
   2.56956555066009,
   -2.58963549738436,
   1.7299541605221),
  (-2.7640879119748,
   -0.95974186620614,
   1.83846475131025,
   -2.3501174879412,
   2.57692876392087),
  (6.24353061

In [85]:
model = pyo.ConcreteModel()

# set: Xi i in [1:K]
#Xi = set(x for x in datasets if x)
# X here is only for one datset 
model.X = pyo.Set(initialize=datasets_tuples) 

In [35]:
model.pprint()

1 Set Declarations
    X : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :   150 :    Any :    1 : {(-0.942246097974701, -0.953678910579824, 0.526591488506912, 3.66132837186367, -0.677236781753973, -1.88198134799196, 6.81334986582624, -5.97394232066749, 2.56726083769774, -0.620779154202402, 1.37128913409494, 1.82482211319299, 0.755575721268344, 0.660719107466978, 1.35370339254185, -2.67426361027708, -0.62697517491745, 2.14869527611797, 1.76778529395218, 3.62247576909984, -1.23812597490318, 3.06385889868078, -1.17535390444881, 0.679876082546988, 2.79042931348587, 1.92574259345985, -0.0011308987749669, 2.52357871754301, -2.20782816281937, 0.508628423065703, 0.707296844939545, 7.90969125550165, -0.907651392639067, -3.70565885426971, -2.91534013418791, -0.637958875665653, -3.67256885396648, -1.69297940738413, -1.23466404487747, -0.980866091211246, 0.116525429157626, -0.71499996643548, -1.04489500813718, -1.39581952818227, 3.37807151560495

In [86]:
# params: ci i in [1:K]
model.c = pyo.Param(model.X, initialize=penalties)

In [87]:
# variables: wi i in [1:K]
# each wi needs to be a vector 1*len(features) -> features is amount of columns in Xi use list 
model.w = pyo.Var(model.X, initialize=init_w)

In [88]:
model.w.pprint()

w : Size=2, Index=X
    Key                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [90]:
model.pprint()

1 Set Declarations
    X : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :   125 :    Any :    2 : {(-0.334610924204471, -0.97306366867377, -0.075583651455657, -0.153609815415446, -0.851993676462297, 5.55820588710131, -3.93991158036192, -6.71995778712731, 5.28723385899056, -0.592665154219762, -1.41301591985501, 1.22647353788828, 2.84067502332716, 0.558083501810207, 4.77974893277835, -6.21790636329919, -0.855823906305692, 4.28703572371224, -3.52227004424225, 2.55882984237342, 5.13895331423074, 1.21981618321478, -0.190048289107061, -0.195436562711887, 1.36919618440577, -8.64338469215024, -1.84681258887914, -0.738421117287049, 5.89215357987772, -7.57379581603491, -9.68340523351363, -6.97307664418281, -6.82514186354428, 5.18311020964875, 0.245243568811294, 1.91891828264879, 1.18361082442049, 2.56956555066009, -2.58963549738436, 1.7299541605221, -2.7640879119748, -0.95974186620614, 1.83846475131025, -2.3501174879412, 2.57692876392087, 6.24

In [94]:
# Objective
model.Obj = pyo.Objective(rule=ObjRule, sense=pyo.maximize)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
ERROR:pyomo.core:Rule failed when generating expression for Objective Obj with index None:
AttributeError: '_GeneralVarData' object has no attribute 'T'
ERROR:pyomo.core:Constructing component 'Obj' from data=None failed:
AttributeError: '_GeneralVarData' object has no attribute 'T'


AttributeError: '_GeneralVarData' object has no attribute 'T'

In [382]:
weigths = pyo.value(model.w[xi])

ERROR:pyomo.core:evaluating object as numeric value: w[-0.942246097974701,-0.953678910579824,0.526591488506912,3.66132837186367,-0.677236781753973,-1.88198134799196,6.81334986582624,-5.97394232066749,2.56726083769774,-0.620779154202402,1.37128913409494,1.82482211319299,0.755575721268344,0.660719107466978,1.35370339254185,-2.67426361027708,-0.62697517491745,2.14869527611797,1.76778529395218,3.62247576909984,-1.23812597490318,3.06385889868078,-1.17535390444881,0.679876082546988,2.79042931348587,1.92574259345985,-0.0011308987749669,2.52357871754301,-2.20782816281937,0.508628423065703,0.707296844939545,7.90969125550165,-0.907651392639067,-3.70565885426971,-2.91534013418791,-0.637958875665653,-3.67256885396648,-1.69297940738413,-1.23466404487747,-0.980866091211246,0.116525429157626,-0.71499996643548,-1.04489500813718,-1.39581952818227,3.37807151560495,-0.872823887518324,0.375732716393443,1.89484952155384,-1.13827884657228,8.44655564978189,0.530175408826722,-1.89297983225569,-1.1377236187841

ValueError: No value for uninitialized NumericValue object w[-0.942246097974701,-0.953678910579824,0.526591488506912,3.66132837186367,-0.677236781753973,-1.88198134799196,6.81334986582624,-5.97394232066749,2.56726083769774,-0.620779154202402,1.37128913409494,1.82482211319299,0.755575721268344,0.660719107466978,1.35370339254185,-2.67426361027708,-0.62697517491745,2.14869527611797,1.76778529395218,3.62247576909984,-1.23812597490318,3.06385889868078,-1.17535390444881,0.679876082546988,2.79042931348587,1.92574259345985,-0.0011308987749669,2.52357871754301,-2.20782816281937,0.508628423065703,0.707296844939545,7.90969125550165,-0.907651392639067,-3.70565885426971,-2.91534013418791,-0.637958875665653,-3.67256885396648,-1.69297940738413,-1.23466404487747,-0.980866091211246,0.116525429157626,-0.71499996643548,-1.04489500813718,-1.39581952818227,3.37807151560495,-0.872823887518324,0.375732716393443,1.89484952155384,-1.13827884657228,8.44655564978189,0.530175408826722,-1.89297983225569,-1.1377236

In [383]:
# constraints: lasso 
model.constraint_lasso = pyo.ConstraintList()
#model.constraint_lasso.add(model.X, rule=lasso) 
for xi in model.X:
        #lasso
        model.constraint_lasso.add(sum(model.w[xi][f] for f in range(len(model.w[xi])) <= model.c[xi]))

TypeError: '_GeneralVarData' object cannot be interpreted as an integer

In [384]:
# constraints: (2-norm)^2 ||wi||22 <=1
model.constraint_norm2 = pyo.ConstraintList()
#model.constraint_norm2.add(model.X, rule=norm2)
for xi in model.X:
        #norm2
    model.constraint_norm2.add(sum(model.w[xi][f] * model.w[xi][f] for f in range(len(model.w[xi])) <= 1))


TypeError: object of type '_GeneralVarData' has no len()

In [385]:
opt = pyo.SolverFactory('glpk')
instance = model.create_instance()
res = opt.solve(instance)

model.solutions.load_from(res)
weights = [wi for wi in instance.w]

return weights

ApplicationError: No executable found for solver 'glpk'